In [3]:
from rag_module.rag import RAG

In [4]:
rag = RAG(LLM_name="openai:gpt-4o-mini")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...


In [8]:
rag.embedder.chunkdata_path

'./embeddings/ELOQ_news/chunk_data.json'

In [3]:
rag.reload_embeddings()

Batches: 100%|██████████| 28/28 [00:08<00:00,  3.31it/s]


In [9]:
rag.query("What is the best airport in north america?")

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.22it/s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'chunks': [{'filename': 'news_2',
   'text': 'This Southern California airport is the best in North America, survey says\n\nTravel in and out of the densely populated region of Southern California can be hectic — but that didn’t stop one of its busiest airports from being named top in North America. According to the 2024 North American Airport Satisfaction Survey from JD Power, Orange County’s John Wayne Airport is the best large airport in North America. The “large airport” category includes airports that receive between 10 and 32.9 million passengers per year. Orange County’s lone commercial airport beat out competitors like Tampa International Airport (No. 2), Kansas City International Airport (No. 3), Dallas Love Field (No. 4) and Nashville International Airport (No. 5). Other California airports ranked in that category include Sacramento International Airport (No. 7), San Jose International Airport (No. 10), Oakland International Airport (No. 19) and San Diego International Airpo